In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from zipfile import ZipFile
home = '/content/drive/My Drive/Datasets/'

with ZipFile(home + "IDAO 2020.zip", 'r') as zip:
  zip.extractall(path="/content/")


In [0]:
import pandas as pd
data = pd.read_csv("/content/IDAO 2020/train.csv")

columns_y = ["x", "y", "z", "Vx", "Vy", "Vz"]
columns_x = ["x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"]

In [7]:
X = data[columns_x]
X

,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237
...,...,...,...,...,...,...
649907,-20717.958996,-16245.240500,5250.939232,-1.653931,3.157321,0.079069
649908,-22673.444496,-11192.339393,5243.608790,-0.945328,3.603371,-0.092202
649909,-23461.830699,-5570.167175,4966.813869,-0.087089,3.912550,-0.281989
649910,-22858.679929,373.249102,4396.055679,0.920162,4.021955,-0.485364


In [8]:
Y = data[columns_y]
Y

,x,y,z,Vx,Vy,Vz
0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083
1,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796
2,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155
3,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115
4,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703
...,...,...,...,...,...,...
649907,-21721.485878,-14048.557595,5277.807430,-1.351754,3.373418,0.004995
649908,-23176.890569,-8712.016936,5153.371350,-0.575955,3.764450,-0.175109
649909,-23363.044794,-2906.071320,4747.247386,0.351381,3.992943,-0.372198
649910,-22058.020262,3074.894039,4038.853542,1.421085,3.984793,-0.578849


In [0]:
#Test train spilt

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.01)

In [0]:
#SVM Regresor

from sklearn import svm
reg = svm.SVR()
reg.fit(X_train[["x_sim"]], Y_train["x"])
pred = reg.predict(X_test[["x_sim"]][:1000])

from sklearn.metrics import *

mse = mean_squared_error(Y_test['x'][:1000], X_test["x_sim"][:1000], sample_weight=None, multioutput='uniform_average', squared=False)

pred = reg.predict(X_test[["x_sim"]][:1000])

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
#Smape Score

import numpy as np
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [0]:
err = smape(X_test["x_sim"][:1000],Y_test['x'][:1000])
err

0.17255281210670645

In [0]:
pred_tmp = Y_test

In [0]:
err = smape(pred,Y_test['x'][:1000])
err

0.3479889725250455

In [0]:
def huber_approx_obj(preds, dtrain):

    d = preds - dtrain #remove .get_labels() for sklearn

    h = 1  #h is delta in the graphic

    scale = 1 + (d / h) ** 2

    scale_sqrt = np.sqrt(scale)

    grad = d / scale_sqrt

    hess = 1 / scale / scale_sqrt

    return grad, hess



In [38]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

xgb_model = {}
y_pred = {}
mse = {}
xgb_model["x"] = xgb.XGBRegressor(objective=huber_approx_obj, random_state=42)

xgb_model["x"].fit(X_train, Y_train["x"])

y_pred["x"] = xgb_model["x"].predict(X_test)

mse["x"]=mean_squared_error(Y_test["x"], y_pred["x"])

print(np.sqrt(mse["x"]))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[14:27:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1526802.586294818


In [39]:
smape(Y_test["x"], y_pred["x"])

0.9851911662691913

In [15]:
xgb_model["y"] = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model["y"].fit(X_train, Y_train["y"])

y_pred["y"] = xgb_model["y"].predict(X_test)

mse["y"]=mean_squared_error(Y_test["y"], y_pred["y"])

print(np.sqrt(mse["y"]))
smape(Y_test["x"], y_pred["x"])

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


6747.398951402705


In [16]:
xgb_model["z"] = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model["z"].fit(X_train, Y_train["z"])

y_pred["z"] = xgb_model["z"].predict(X_test)

mse["z"]=mean_squared_error(Y_test["z"], y_pred["z"])

print(np.sqrt(mse["z"]))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


5872.67035393267


In [17]:
xgb_model["Vy"] = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model["Vy"].fit(X_train, Y_train["Vy"])

y_pred["Vy"] = xgb_model["Vy"].predict(X_test)

mse["Vy"]=mean_squared_error(Y_test["Vy"], y_pred["Vy"])

print(np.sqrt(mse["Vy"]))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


1.292417335608456


In [18]:
xgb_model["Vx"] = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model["Vx"].fit(X_train, Y_train["Vx"])

y_pred["Vx"] = xgb_model["Vx"].predict(X_test)

mse["Vx"]=mean_squared_error(Y_test["Vx"], y_pred["Vx"])

print(np.sqrt(mse["Vx"]))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


1.2137879973398864


In [19]:
xgb_model["Vz"] = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model["Vz"].fit(X_train, Y_train["Vz"])

y_pred["Vz"] = xgb_model["Vz"].predict(X_test)

mse["Vz"]=mean_squared_error(Y_test["Vz"], y_pred["Vz"])

print(np.sqrt(mse["Vz"]))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


1.0161959980197268


In [0]:
err = smape(y_pred,Y_test['x'])
err

0.2028860103900693

In [0]:
sub = pd.read_csv("/content/IDAO 2020/test.csv")

In [0]:
data_test_1= pd.read_csv("/content/IDAO 2020/Track 1/test.csv")

In [0]:
data_test = data_test_1[columns_x]

In [22]:
data_test

,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
1,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
2,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
4,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724
...,...,...,...,...,...,...
284066,28595.031634,-85458.623976,5021.767767,-1.721131,-0.011611,0.122357
284067,15547.173728,-84233.509948,5840.616702,-1.815323,0.353445,0.098927
284068,1981.107111,-80123.860051,6458.394351,-1.858963,0.773846,0.067379
284069,-11644.801379,-72674.504171,6805.361999,-1.826030,1.262930,0.024782


In [0]:
test_pred = {}
test_pred["x"] =  xgb_model["x"].predict(data_test)

In [0]:
test_pred["y"] =  xgb_model["y"].predict(data_test)
test_pred["z"] =  xgb_model["z"].predict(data_test)
test_pred["Vx"] =  xgb_model["Vx"].predict(data_test)
test_pred["Vy"] =  xgb_model["Vy"].predict(data_test)
test_pred["Vz"] =  xgb_model["Vz"].predict(data_test)

In [26]:
len(test_pred["Vx"])

284071

In [30]:
test_pred["id"] = data_test_1["id"]
test_pred

{'Vx': array([ 4.0060186,  4.5537224,  4.5189834, ..., -1.818569 , -1.7019246,
        -1.5727873], dtype=float32),
 'Vy': array([-0.6352428,  0.5566545,  2.123705 , ...,  0.5413923,  1.1232002,
         1.6557292], dtype=float32),
 'Vz': array([ 0.83950645,  0.4543083 , -0.3099972 , ...,  0.08406773,
        -0.01775801, -0.0437454 ], dtype=float32),
 'id': 0            3927
 1            3928
 2            3929
 3            3930
 4            3931
            ...   
 284066    1231060
 284067    1231061
 284068    1231062
 284069    1231063
 284070    1231064
 Name: id, Length: 284071, dtype: int64,
 'x': array([-12950.15   ,  -6236.6416 ,    329.85468, ...,   3963.0703 ,
         -8476.799  , -19683.484  ], dtype=float32),
 'y': array([-13176.925, -13795.901, -12948.886, ..., -79214.79 , -71806.2  ,
        -61122.996], dtype=float32),
 'z': array([5573.8247, 5804.7134, 6005.047 , ..., 5871.0884, 6159.8613,
        6159.8613], dtype=float32)}

In [31]:
pred_df = pd.DataFrame(test_pred)
pred_df

,x,y,z,Vx,Vy,Vz,id
0,-12950.150391,-13176.924805,5573.824707,4.006019,-0.635243,0.839506,3927
1,-6236.641602,-13795.901367,5804.713379,4.553722,0.556655,0.454308,3928
2,329.854675,-12948.885742,6005.046875,4.518983,2.123705,-0.309997,3929
3,5225.054199,-7449.567871,4867.064453,3.986324,4.270105,-1.213306,3930
4,8225.722656,-1165.880005,3771.588135,0.364042,4.345695,-2.219078,3931
...,...,...,...,...,...,...,...
284066,30678.417969,-83456.765625,4871.836914,-1.507415,-0.071196,0.105396,1231060
284067,17369.640625,-80307.234375,4871.836914,-1.808297,0.259418,0.105396,1231061
284068,3963.070312,-79214.789062,5871.088379,-1.818569,0.541392,0.084068,1231062
284069,-8476.798828,-71806.203125,6159.861328,-1.701925,1.123200,-0.017758,1231063


In [32]:
pred_df = pred_df[["id", "x", "y", "z", "Vx", "Vy", "Vz"]]
pred_df

,id,x,y,z,Vx,Vy,Vz
0,3927,-12950.150391,-13176.924805,5573.824707,4.006019,-0.635243,0.839506
1,3928,-6236.641602,-13795.901367,5804.713379,4.553722,0.556655,0.454308
2,3929,329.854675,-12948.885742,6005.046875,4.518983,2.123705,-0.309997
3,3930,5225.054199,-7449.567871,4867.064453,3.986324,4.270105,-1.213306
4,3931,8225.722656,-1165.880005,3771.588135,0.364042,4.345695,-2.219078
...,...,...,...,...,...,...,...
284066,1231060,30678.417969,-83456.765625,4871.836914,-1.507415,-0.071196,0.105396
284067,1231061,17369.640625,-80307.234375,4871.836914,-1.808297,0.259418,0.105396
284068,1231062,3963.070312,-79214.789062,5871.088379,-1.818569,0.541392,0.084068
284069,1231063,-8476.798828,-71806.203125,6159.861328,-1.701925,1.123200,-0.017758


In [0]:
pred_df.to_csv("/content/IDAO 2020/Track 1/sub2.csv")